In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#get the data from
!wget https://www.dropbox.com/scl/fi/wzy3syorl7cdkbl9dvt2w/archive.zip?rlkey=igq81bgfgo2uagkpvl6rr2duw&st=eeoxypvx&dl=0

--2024-10-23 16:50:08--  https://www.dropbox.com/scl/fi/wzy3syorl7cdkbl9dvt2w/archive.zip?rlkey=igq81bgfgo2uagkpvl6rr2duw
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uceae3b8b25c8519be4820e68bf7.dl.dropboxusercontent.com/cd/0/inline/CdCMV8pI-vFrCYR4SDR72x3Q7xV0FN5erz4Q2C_-ACKPIXeulubEonDXwzHlfvXqdpdm_FzCpetRD9MKJL_lab2IUC6rGdoz_YfhuZlNhZyN36sQUgXAeBvfWbBvC9gF5ANgKinr3QaSyUtZdGKYwZb0/file# [following]
--2024-10-23 16:50:10--  https://uceae3b8b25c8519be4820e68bf7.dl.dropboxusercontent.com/cd/0/inline/CdCMV8pI-vFrCYR4SDR72x3Q7xV0FN5erz4Q2C_-ACKPIXeulubEonDXwzHlfvXqdpdm_FzCpetRD9MKJL_lab2IUC6rGdoz_YfhuZlNhZyN36sQUgXAeBvfWbBvC9gF5ANgKinr3QaSyUtZdGKYwZb0/file
Resolving uceae3b8b25c8519be4820e68bf7.dl.dropboxusercontent.com (uceae3b8b25c8519be4820e68bf7.dl.dropboxusercontent.com)... 162.125.81.15, 262

In [ ]:
!unzip /content/archive.zip?rlkey=igq81bgfgo2uagkpvl6rr2duw

Archive:  /content/archive.zip?rlkey=igq81bgfgo2uagkpvl6rr2duw
replace Testing/glioma/Te-glTr_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Testing/glioma/Te-glTr_0000.jpg  
replace Testing/glioma/Te-glTr_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace Testing/glioma/Te-glTr_0001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Testing/glioma/Te-glTr_0001.jpg  
replace Testing/glioma/Te-glTr_0002.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: Testing/glioma/Te-glTr_0002.jpg  
replace Testing/glioma/Te-glTr_0003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob


In [ ]:
TRAIN_DIR = "/content/Training"
number_of_images = {}

for dir in os.listdir(TRAIN_DIR):
  number_of_images[dir]= len(os.listdir(os.path.join(TRAIN_DIR,dir)))
number_of_images.items()

dict_items([('glioma', 1321), ('pituitary', 1457), ('notumor', 1595), ('meningioma', 1339)])

In [ ]:

TEST_DIR = "/content/Testing"
number_of_images = {}

for dir in os.listdir(TEST_DIR):
  number_of_images[dir]= len(os.listdir(os.path.join(TEST_DIR,dir)))
number_of_images.items()

dict_items([('glioma', 300), ('pituitary', 300), ('notumor', 405), ('meningioma', 306)])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
def preprocessingimages(path):


  #image_data = ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2, rescale=1./255, horizontal_flip=True)
  #For a more generalized data set
  image_data = ImageDataGenerator(preprocessing_function= preprocess_input)
  image = image_data.flow_from_directory(directory= path, target_size=(512,512), batch_size=32, class_mode= 'categorical',color_mode='grayscale')
  return image


In [ ]:
path='/content/Training'
train_data = preprocessingimages(path)

Found 5712 images belonging to 4 classes.


In [ ]:
path='/content/Testing'
test_data = preprocessingimages(path)

Found 1311 images belonging to 4 classes.


Model Training using transfer learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
import keras
from tensorflow.keras.layers import InputLayer
import tensorflow as tf

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
def convert_grayscale_to_rgb(image):
    return np.stack((image,) * 3, axis=-1)

# ImageDataGenerator with preprocessing function to convert grayscale to RGB
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load your train and test data
train_data = train_datagen.flow_from_directory(
    '/content/Training',
    target_size=(512, 512),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    '/content/Testing',
    target_size=(512, 512),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical'
)

# Create MobileNet model with the correct input shape
base_model = MobileNet(input_shape=(512, 512, 3), include_top=False)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

X= Flatten()(base_model.output)
X= Dense (units= 4, activation='softmax')(X)

model= Model(base_model.input,X)

#base_model.summary()


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 512, 512, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 256, 256, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 256, 256, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 256, 256, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 256, 256, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 256, 256, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 256, 256, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 256, 256, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 257, 257, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 128, 128, 64)        │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 128, 128, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 128, 128, 128)       │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 128, 128, 128)       │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 128, 128, 128)       │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 128, 128, 128)       │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 128, 128, 128)       │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 4,277,444 (16.32 MB)

 Trainable params: 1,048,580 (4.00 MB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [ ]:
model.compile(optimizer='adam',loss= keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc= ModelCheckpoint(filepath='bestmodel.keras', monitor='val_accuracy', save_best_only=True)
es= EarlyStopping( monitor='val_accuracy', min_delta=0.01,patience= 3, verbose=1)
cb= [mc,es]

In [ ]:
hist= model.fit(train_data, steps_per_epoch=2,
                epochs= 30, validation_data= test_data, callbacks=cb)

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.4440 - loss: 41.7798 - val_accuracy: 0.7918 - val_loss: 8.6637
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.8351 - loss: 7.8832 - val_accuracy: 0.8520 - val_loss: 6.8972
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 17s 880ms/step - accuracy: 0.9116 - loss: 3.1032 - val_accuracy: 0.8757 - val_loss: 4.0160
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 772ms/step - accuracy: 0.8940 - loss: 3.0695 - val_accuracy: 0.8894 - val_loss: 4.5705
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 776ms/step - accuracy: 0.9369 - loss: 2.7777 - val_accuracy: 0.9123 - val_loss: 2.9340
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 786ms/step - accuracy: 0.9293 - loss: 2.0980 - val_accuracy: 0.9191 - val_loss: 2.8071
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 779ms/step - accuracy: 0.9409 - loss: 3.1527 - val_accuracy: 0.9329 - val_loss: 2.4014
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 763ms/step - accuracy: 0.9603 - loss: 1.1637 - val_accuracy:

Load best fit model

In [ ]:
model = load_model("/content/bestmodel.keras")

In [ ]:
acc = model.evaluate(test_data)[1]
print(f"Model accuracy is {acc*100}%")

In [ ]:
hist.history.keys()

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'], c = "red")
plt.title("model accuracy")
plt.ylabel("acc")
plt.xlabel("epochs")

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'], c = "red")
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epochs")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

path = "/content/Testing/pituitary/Te-piTr_0002.jpg"
img = image.load_img(path, target_size=(512,512))
i = image.img_to_array(img)/255
im = preprocess_input(i)

input_arr = np.array([im])
input_arr.shape
pred = np.argmax(model.predict(input_arr))

class_names = ['glioma', 'meningioma', 'no tumor', 'pituitary']


In [ ]:
# @title
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.title(f"Predicted Class: {class_names[pred]}")
plt.axis('off')  # Hide axes
plt.show()